In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from tensorboardX import SummaryWriter      

import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "vgg_quant_ProjectPart1"
model = VGG16_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
# HW

#  1. Load your saved model and validate
#  2. Replace your model's all the Conv's weight with quantized weight
#  3. Apply reasonable alpha
#  4. Then, try to multiple bit precisions and draw graph of bit precision vs. accuracy

In [3]:
PATH = "result/ProjectPart1Model/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9035/10000 (90%)



In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
41 -th layer prehooked
46 -th layer prehooked
50 -th layer prehooked
54 -th layer prehooked


In [5]:
weight_q = model.features[27].weight_q
w_alpha = model.features[27].weight_quant.wgt_alpha
w_bit = 4
w_delta = w_alpha/((2**(w_bit-1)-1))
weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))
#print(weight_int)
print(weight_int.abs().sum())

tensor(1484., device='cuda:0', grad_fn=<SumBackward0>)


In [6]:
act = save_output.outputs[8][0]
act_alpha = model.features[27].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)
act_delta = act_alpha/((2**act_bit)-1)
act_q = act_quant_fn(act, act_alpha)
#print(act_int)
act_int = act_q / (act_alpha / (2**act_bit-1))
print(act_int.abs().sum())

tensor(24156., device='cuda:0', grad_fn=<SumBackward0>)


In [7]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1)
conv_int.weight = torch.nn.Parameter(weight_int)
conv_int.bias = model.features[27].bias #type(model.features[3].bias) = NoneType
output_int = conv_int(act_int)
output_recovered = output_int * (act_alpha / (2**act_bit-1)) * (w_alpha / (2**(w_bit-1)-1))
#print(output_recovered)
#print(output_recovered[0])

In [8]:
conv_ref = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1)
conv_ref.weight = model.features[27].weight_q
conv_ref.bias = model.features[27].bias #type(model.features[3].bias) = NoneType
output_ref = conv_ref(act)
#print(output_ref)
#print(output_ref[0]

In [9]:
import math

# act_int.size = torch.Size([128, 8, 4, 4]) <- batch_size, input_ch, ni, nj
a_int = act_int[0,:,:,:] # pick only one input out of batch
# act_int.size() = torch.Size([128, 8, 4, 4])
# a_int.size() = [8, 4, 4]
# conv_int.weight.size() = torch.Size([8, 8, 3, 3]) <- output_ch, input_ch, ki, kj
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1),-1))
# merge ki, kj index to kij
# w_int.weight.size() = torch.Size([64, 64, 9])
padding = 1
stride = 1
array_size = 8 # row and column number
######## Inputs ########
nig = range(a_int.size(1)) ## ni group [0,1,...31]
njg = range(a_int.size(2)) ## nj group
nijg = range(a_int.size(1)*a_int.size(2))
######## Weights and related stuff ########
kijg = range(w_int.size(2)) # [0, .. 8]
ki_dim = int(math.sqrt(w_int.size(2))) ## Kernel's 1 dim size = 3
kig = range(int(math.sqrt(len(kijg)))) ## = 3
kjg = range(int(math.sqrt(len(kijg)))) ## = 3
######## Channels ########
icg = range(int(w_int.size(1))) ## input channel [0,...7]
ocg = range(int(w_int.size(0))) ## output channel [0,...7]
ic_tileg = range(int(len(icg)/array_size)) ##[0]
oc_tileg = range(int(len(ocg)/array_size)) ##[0]
######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(njg)+padding*2).cuda()
# a_pad.size() = [8, 4+2pad, 4+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0),-1)) ## merge ni and nj index into nij
# a_pad.size() = [8, (4+2pad)*(4+2pad)]
a_tile = torch.zeros(len(ic_tileg), len(oc_tileg), array_size, a_pad.size(1)).cuda()
for ic_tile in ic_tileg: #spatial
    for oc_tile in oc_tileg: #spatial
        a_tile[ic_tile,oc_tile,:,:] = a_pad[ic_tile*array_size:(ic_tile+1)*array_size,:]
p_nijg = range(a_tile.size(3)) ## paded activation's nij group [0, ...34*34-1]

######## Outputs ########
o_nig = range(int((math.sqrt(len(nijg))+2*padding-(math.sqrt(len(kijg))- 1)- 1)/stride + 1)) #range(0, 32)
o_njg = range(int((math.sqrt(len(nijg))+2*padding-(math.sqrt(len(kijg))- 1)- 1)/stride + 1)) #range(0, 32)
psum=torch.zeros(len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)).cuda()
out = torch.zeros(len(ocg), len(o_nig), len(o_njg)).cuda()
######## Tiled 2D version ########
for ic_tile in ic_tileg: #spatial
    for oc_tile in oc_tileg: #spatial
        for kij in kijg: #temporal
            m = nn.Linear(array_size, array_size, bias=False)
            m.weight = torch.nn.Parameter(w_int[oc_tile*array_size: (oc_tile+1)*array_size,ic_tile*array_size:(ic_tile+1)*array_size,kij])
            for nij in p_nijg: #temporal
                psum[ic_tile, oc_tile, :, nij, kij] = m(a_tile[ic_tile,oc_tile,:, nij]).cuda()

### SFP accumulation ###
for ni in o_nig:
    for nj in o_njg:
        for ki in kig:
            for kj in kjg:
                for ic_tile in ic_tileg:
                    for oc_tile in oc_tileg:
                        out[oc_tile*array_size:(oc_tile+1)*array_size, ni, nj] = out[oc_tile*array_size:(oc_tile+1)*array_size, ni, nj] + \
                        psum[ic_tile, oc_tile, :, int(math.sqrt(len(p_nijg)))*(ni+ki) + (nj+kj), len(kig)*ki+kj]
out_relu = out.clone()

### RELU ###
for ni in range(out.size()[1]):
    for nj in range(out.size()[2]):
        for oc in range(out.size()[0]):
            out_relu[oc,ni,nj]=out[oc,ni,nj]*(round(out[oc,ni,nj].item())>0)
 

In [10]:
#This cell shows how recovered output has low error as compared to actual output
(save_output.outputs[9][0][0]- out_relu.mul(act_delta).mul(w_delta)).abs().sum()

tensor(2.4930e-05, device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
### show this cell partially. The following cells should be printed by students ###
ic_tile_id = 0
oc_tile_id = 0
#nij = 200 # just a random number
X = a_tile[ic_tile_id,oc_tile_id,:,:] # [tile_num, array row num, time_steps]

bit_precision = 4
file = open('activation_tile0.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)): # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(round(X[array_size-j-1,i].item()))
        for k in range(bit_precision):
            file.write(X_bin[k])
        #file.write(' ') # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file

In [ ]:
### show this cell partially. The following cells should be printed by students ###
#w_int
bit_precision = 4

for kij in range(w_int.size(2)): # kij
    file_name = "weight_itile0_otile0_kij{}.txt".format(kij)
    file = open(file_name, 'w') #write to file
    file.write('#ic7oc0[msb-lsb],ic6oc0[msb-lst],....,ic0oc0[msb-lst]#\n')
    file.write('#ic7oc1[msb-lsb],ic6oc1[msb-lst],....,ic0oc1[msb-lst]#\n')
    file.write('#................#\n')
    for oc in range(w_int.size(0)): # output channel
        for ic in range(w_int.size(1)): # input channel
            w_int_bin = '{0:04b}'.format(round(w_int[oc,array_size-ic-1,kij].item())+2**(bit_precision)*(round(w_int[oc,array_size-ic-1,kij].item())<0))
            for k in range(bit_precision):
                file.write(w_int_bin[k])
        file.write('\n')
        #file.write(' ') # for visibility with blank between words, you can use
file.close() #close file


In [ ]:
### show this cell partially. The following cells should be printed by students ###
#w_int
bit_precision = 16
file = open('out.txt', 'w') #write to file
file.write('#oc0nij0[msb-lsb],oc1nij0[msb-lst],....,oc7nij0[msb-lst]#\n')
file.write('#oc0nij1[msb-lsb],oc1nij1[msb-lst],....,oc7nij1[msb-lst]#\n')
file.write('#................#\n')

for ni in range(out.size(1)): # ni
    for nj in range(out.size(2)): # nj
        for oc in range(out.size(0)): # output channel
            out_int_bin = '{0:016b}'.format(round(out[oc,ni,nj].item())+2**(bit_precision)*(round(out[oc,ni,nj].item())<0))
            for k in range(bit_precision):
                file.write(out_int_bin[k])
        file.write('\n')
#file.write(' ') # for visibility with blank between words, you can use
file.close() #close file

In [ ]:
 ### show this cell partially. The following cells should be printed by students ###
 #w_int
bit_precision = 16
file = open('out_relu.txt', 'w') #write to file
file.write('#oc0nij0[msb-lsb],oc1nij0[msb-lst],....,oc7nij0[msb-lst]#\n')
file.write('#oc0nij1[msb-lsb],oc1nij1[msb-lst],....,oc7nij1[msb-lst]#\n')
file.write('#................#\n')

for ni in range(out_relu.size(1)): # ni
    for nj in range(out_relu.size(2)): # nj
        for oc in range(out_relu.size(0)): # output channel
            out_int_bin = '{0:016b}'.format(round(out_relu[oc,ni,nj].item())+2**(bit_precision)*(round(out_relu[oc,ni,nj].item())<0))
            for k in range(bit_precision):
                file.write(out_int_bin[k])
        file.write('\n')
#file.write(' ') # for visibility with blank between words, you can use
file.close() #close file

In [ ]:
len_kij   = 9      # number of kernel taps (kij = 0..8)
len_onij  = 16     # number of output positions (4x4)
addr_bits = 11     # width of PMEM address (A_pmem is 11 bits)

with open("acc.txt", "w") as f:
    # Header lines (same style as before)
    f.write("#acc0: addresses of psums to be accumulated per output window#\n")
    f.write("#9 addresses (for kij 0..8) per (ni,nj) output position#\n")
    f.write("#................#\n")

    # For each output window index i (0..15)
    for out_idx in range(len_onij):  # 0..15
        # For each kij (0..8), compute pmem address = kij*16 + out_idx
        for kij in range(len_kij):
            addr = kij * len_onij + out_idx    # kij*16 + out_idx
            addr_bin = format(addr, f"0{addr_bits}b")  # 11-bit binary
            f.write(addr_bin + "\n")

file.close()
print("acc.txt generated with", (len(o_nig) * len(o_njg) * len(kig) * len(kjg)), "addresses")


acc.txt generated with 144 addresses


In [16]:
act = save_output.outputs[9][0]
act_alpha = model.features[27].act_alpha #maybe use previous layers act alpha only
act_bit = 4
act_quant_fn = act_quantization(act_bit)
act_q = act_quant_fn(act, act_alpha)
#print(act_int)
act_int = act_q / (act_alpha / (2**act_bit-1))
print(act_int.abs().sum())

tensor(102665., device='cuda:0', grad_fn=<SumBackward0>)
